In [ ]:
# Import
import os
import pandas as pd
import numpy as np

from pathlib import Path

from CellPacking.tissuegeneration import sheet_init, symetric_circular
from CellPacking.dynamics import (Compression, 
                                  AnisotropicLineTension, 
                                  ShearPlanarGeometry, 
                                  PlaneBarrierElasticity)

from tyssue import Sheet
from tyssue import PlanarGeometry
from tyssue.solvers import QSSolver
from tyssue.solvers.viscous import EulerSolver
from tyssue.behaviors.event_manager import EventManager
from tyssue.behaviors.sheet.basic_events import reconnect, reconnect_3D
from tyssue.dynamics import model_factory, effectors
from tyssue.core.history import HistoryHdf5 


import matplotlib.pyplot as plt
from tyssue.draw import sheet_view
from CellPacking.plot import superimpose_sheet_view
from CellPacking.plot import sheet_view as ply_sheet_view

from tyssue.generation import extrude 
from tyssue import Monolayer
from CellPacking.dynamics import ShearMonolayerGeometry
from tyssue.io.hdf5 import save_datasets

from tyssue.io.hdf5 import load_datasets
from tyssue.io.meshes import save_triangular_mesh

In [ ]:
SIM_DIR = Path('/mnt/sda1/Sophie/0-Simulations/20221015_3D_QS_without_HI')
sim_save_dir = SIM_DIR
try:
    os.mkdir(sim_save_dir)
except FileExistsError:
    pass

In [ ]:
%pdb

# Apical sheet init

In [ ]:
def tissue_init(phi, noise):
    
    apical_sheet, border = symetric_circular(10, -100, phi, 0,  noise=noise)

    apical_sheet.face_df['prefered_perimeter'] = 3 * np.sqrt(apical_sheet.face_df['prefered_area'])

    ## apical surface at the equilibrium
    # Solver
    solver_qs = QSSolver(with_t1=False, with_t3=False, with_collisions=False)

    manager = EventManager()
    manager.append(reconnect)
    # Model
    model = model_factory(
        [
            effectors.FaceAreaElasticity,
            effectors.PerimeterElasticity, 
        ])
    for i in range(50):
        manager.execute(apical_sheet)
        res = solver_qs.find_energy_min(apical_sheet, PlanarGeometry, model, periodic=False, options={"gtol": 1e-8})
        if res.success is False:
            print (i, res.success)
        apical_sheet.vert_df[["x", "y"]] += np.random.normal(scale=1e-3, size=(apical_sheet.Nv, 2))
        PlanarGeometry.update_all(apical_sheet)
        manager.update()


    ## Monolayer creation
    apical_sheet.face_df['z'] = 1
    apical_sheet.edge_df['z'] = 1
    apical_sheet.vert_df['z'] = 1

    extruded = extrude(apical_sheet.datasets, method='translation', vector=[0, 0, -2])
    monolayer = Monolayer('mono', extruded)

    monolayer.sanitize(trim_borders=True, order_edges=True)
    monolayer.validate()


    monolayer.face_df['prefered_area'] = monolayer.face_df.loc[0,'prefered_area']
    monolayer.face_df['prefered_perimeter'] = 3*np.sqrt(monolayer.face_df['prefered_area'])
    monolayer.face_df['area_elasticity'] = 1
    monolayer.face_df['perimeter_elasticity'] = 0.5


    

    monolayer.vert_df['barrier_elasticity'] = 280
    monolayer.vert_df['z_barrier'] = 0.6

    monolayer.update_specs({"settings":{"dt":0.01,
                                         'threshold_length': 0.1,
                                         'p_4': 1,
                                         'p_5p': 1,
                                          "nrj_norm_factor": 1.0, 
                                       'multiplier':3, },
                          "cell": {
                                    "x": 0.0,
                                    "y": 0.0,
                                    "z": 0.0,
                                    "is_alive": True,
                                    "prefered_volume": 0.7,
                                    "volume": 0.7,
                                    "volume_elasticity": 0.5,
                                    "z_barrier":1.1,
                                    },
                          "edge": {
                                    "dx": 0.0,
                                    "srce": 0,
                                    "face": 0,
                                    "dy": 0.0,
                                    "ny": 0.0,
                                    "nx": 0.0,
                                    "length": 0.0,
                                    "nz": 0.0,
                                    "cell": 0,
                                    "sub_volume": 0.0,
                                    "dz": 0.0,
                                    "sub_area": 0.0,
                                    "trgt": 0},
                          "vert": {
                                    "x": 0.0,
                                    "is_active": True,
                                    "z": 0.0,
                                    "y": 0.0},
                          "face": {
                                    "x": 0.0,
                                    "is_alive": True,
                                    "z": 0.0,
                                    "y": 0.0,}
                        })


    ShearMonolayerGeometry.update_all(monolayer)

    monolayer.face_df['prefered_area'] = monolayer.face_df['area']
    monolayer.face_df['prefered_perimeter'] = 3*np.sqrt(monolayer.face_df['prefered_area'])

    ShearMonolayerGeometry.update_all(monolayer)

    # Manager
    manager = EventManager('face')#, track_event=False)

    monolayer.get_opposite_faces()

    monolayer.get_opposite_faces()
    edge_opp_face = monolayer.upcast_face(monolayer.face_df['opposite'])
    monolayer.edge_df['is_border']=False
    monolayer.edge_df.loc[edge_opp_face[edge_opp_face==-1].index, 'is_border']=True

    monolayer.edge_df['opposite'] = pd.to_numeric(monolayer.edge_df['opposite'])
    monolayer.edge_df['z'] = pd.to_numeric(monolayer.edge_df['z'])

    ShearMonolayerGeometry.update_all(monolayer)

    # monolayer equilibrium before apply forces 
    # Model
    model = model_factory(
        [
#                 PlaneBarrierElasticity,
    #         effectors.LineTension,
            effectors.FaceAreaElasticity,
            effectors.PerimeterElasticity,
#                 effectors.CellVolumeElasticity,
        ],
    )
    manager.append(reconnect_3D)
    solver_qs = QSSolver(with_t1=False, with_t3=False, with_collisions=False)

    for i in range(10):
        manager.execute(monolayer)

        res = solver_qs.find_energy_min(monolayer, ShearMonolayerGeometry, model, periodic=False, options={"gtol": 1e-8})
        if res.success is False:
            print (i, res.success)
        monolayer.vert_df[["x", "y"]] += np.random.normal(scale=1e-3, size=(monolayer.Nv, 2))
        save_triangular_mesh('monolayer0.19.vtk', monolayer)
        ShearMonolayerGeometry.update_all(monolayer)
        manager.update()


    return monolayer


def simu_process(monolayer_, r, g):
    
    monolayer=monolayer_.copy(deep_copy=True)
    sim_save_dir = SIM_DIR/str(r)
    try:
        os.mkdir(sim_save_dir)
    except FileExistsError:
        pass
    
    try:
        os.mkdir(sim_save_dir/str(g))
    except FileExistsError:
        pass

    gamma_0 = g
    monolayer.update_specs({"edge": {"gamma_0": g}})
    monolayer.edge_df['gamma_0'] = pd.to_numeric(monolayer.edge_df['gamma_0'])
    monolayer.edge_df['gamma_0'] = monolayer.edge_df['gamma_0'].replace(-100, g)
    
    monolayer.specs['settings']['file_text'] = os.path.join(sim_save_dir/str(g),'output_t1.txt')
    
    ## Apply forces
    # Model

    model = model_factory(
        [
#                 PlaneBarrierElasticity,
            effectors.LineTension,
            effectors.FaceAreaElasticity,
            effectors.PerimeterElasticity,
#                 effectors.CellVolumeElasticity,
        ],
    )

    solver_qs = QSSolver(with_t1=False, with_t3=False, with_collisions=False)
    # Manager
    manager = EventManager('face')#, track_event=False)
    manager.append(reconnect_3D)

    for i in range(200):
        print('------------TEMPS------------')
        print(i)
        manager.execute(monolayer)

        res = solver_qs.find_energy_min(monolayer, ShearMonolayerGeometry, model, periodic=False, options={"gtol": 1e-8})
        if res.success is False:
            print (i, res.success)

        monolayer.vert_df[["x", "y"]] += np.random.normal(scale=1e-3, size=(monolayer.Nv, 2))
        manager.update()
        save_datasets(os.path.join(sim_save_dir/str(g),'monolayer'+str(i)+'.hf5'), monolayer)
#         save_triangular_mesh('monolayer0.19.vtk', monolayer)

In [ ]:
from joblib import Parallel, delayed
import multiprocessing
from datetime import datetime


global_start=datetime.now()
print ("start : " + str(global_start))
num_cores = multiprocessing.cpu_count()


repeat = np.arange(10)
gammas = np.linspace(0, 0.2, 21)
# gammas = np.linspace(0.1, 0.2, 11)
# gammas = np.linspace(0.19, 0.19, 1)

# Number of cells in x and y axis
nx = 40
ny = 40
noise = 0.3
phi = np.pi/2

for r in repeat[2:]:
    monolayer = tissue_init(phi, noise)
    
    results = Parallel(n_jobs=6)(delayed(simu_process)(
        monolayer, r, g) for g in gammas)

# simu_process(repeat[0][0], gammas[0][0], phi, noise)
global_end = datetime.now()
print ("end : " + str(global_end))
print ('Duree totale d execution : \n\t\t')
print (global_end-global_start)

In [ ]:
import time
from plyer import notification
notification.notify(
    title = "ALERT!!!",
    message = "It is finish",
    timeout=10
)

In [ ]:
#  raise SystemExit("Stop right there!")


# Analyse

In [ ]:

result = pd.DataFrame(columns = ['repeat', 'gamma', 'nb_change', 'tot_cell'])

repeat = np.arange(10)
gammas = np.linspace(0, 0.2, 21)
# gammas = np.linspace(0.19, 0.19, 1)

for r in repeat:
    sim_save_dir = SIM_DIR/str(r)   
    for g in gammas:
        dir_ = sim_save_dir/str(g)
        try : 
            monolayer_d = load_datasets(os.path.join(sim_save_dir/str(g),'monolayer99.hf5'))
        except: 
            monolayer_d = load_datasets(os.path.join(sim_save_dir/str(g),'monolayer199.hf5'))
        monolayer = Monolayer("mono", monolayer_d)
        count = len(np.unique(monolayer.edge_df[(monolayer.edge_df['face'].isin(monolayer.face_df[monolayer.face_df['num_sides']==3].index)) &
                  (monolayer.edge_df['segment']=='lateral') & (monolayer.edge_df['face'].isin(monolayer.face_df[monolayer.face_df['area']>0.01].index))]['cell']))
        
        result = pd.concat([result, pd.DataFrame({'repeat':r,
                                                  'gamma':g, 
                                                  'nb_change':count,
                                                 'tot_cell': monolayer.Nc},
                          index=[0])],
                          ignore_index=True)
        
        save_triangular_mesh('monolayer'+str(g)+'.vtk', monolayer)
        
result['pourcentage'] = result['nb_change']/result['tot_cell']*100

In [ ]:
result

In [ ]:
fig, ax = plt.subplots()
ax.plot(result['gamma'], result['pourcentage'], '.', markersize=10, color='black')
ax.set_xlabel('gamma')
ax.set_ylabel('% of cell that have neighbouring change')

ax.plot(result.groupby('gamma').mean().index, result.groupby('gamma').mean()['pourcentage'], 
        '.',color='red',  markersize=10, label='mean')
# fig.set_label()
fig.set_size_inches((10,10))

fig.savefig(SIM_DIR/'result.png', dpi=150)

In [ ]:
fig, ax = plt.subplots()
# ax.plot(result['gamma'], result['pourcentage'], '.', markersize=10, color='black')
ax.set_xlabel('gamma')
ax.set_ylabel('% of cell that have neighbouring change')

ax.plot(result.groupby('gamma').mean().index, result.groupby('gamma').mean()['pourcentage'], 
        '.',color='red',  markersize=10, label='mean')


ax.errorbar(result.groupby('gamma').mean().index,
            result.groupby('gamma').mean()['pourcentage'],
            result.groupby('gamma').std()['pourcentage'],
            linestyle='None', fmt='-o', color='red')
fig.set_size_inches((10,10))

# fig.savefig(SIM_DIR/'result.eps', dpi=300)

In [ ]:
result.to_csv(os.path.join(SIM_DIR, 'result_count.csv'))

In [ ]:
from tyssue.io.meshes import save_triangular_mesh
save_triangular_mesh('monolayer.vtk', monolayer)

# Result V2

In [ ]:

result_t1_angle = pd.DataFrame(columns = ['repeat', 'gamma', 'angle'])

repeat = np.arange(10)
gammas = np.linspace(0, 0.2, 21)
# gammas = np.linspace(0.19, 0.19, 1)

for r in repeat:
    sim_save_dir = SIM_DIR/str(r)   
    for g in gammas:
        dir_ = sim_save_dir/str(g)
        try : 
            
            t1_angle = pd.read_csv(os.path.join(dir_,'output_t1.txt'), header=None, sep="\t")
            t1_angle.columns = ['edge_id', 'area', 'angle']
            t1_angle["angle_0_pi"] = [np.pi+a if a<0 else a for a in t1_angle['angle']]
            t1_angle_filter = t1_angle[t1_angle["area"]>0.1]["angle_0_pi"]
            
            result_t1_angle = pd.concat([result_t1_angle, pd.DataFrame({'repeat':r,
                                                      'gamma':g, 
                                                      'angle':t1_angle_filter,
                                                     })],
                          ignore_index=True)
        
        except: 
            pass
        

In [ ]:
# result_t1_angle.to_csv(os.path.join(SIM_DIR, 'result_t1_angle.csv'))

In [ ]:
plt.hist(result_t1_angle[result_t1_angle["gamma"]==0]['angle'], bins=18,alpha=0.5, density=False)
plt.hist(result_t1_angle[result_t1_angle["gamma"]==0.2]['angle'], bins=18, alpha=0.5, density=False)

In [ ]:
b=18
for repeat in range (2, 10):
    fig = plt.figure()
    polar_ax = fig.add_subplot(1, 1, 1, projection="polar")

    # bin data for our polar histogram
    counts0_, bin_0 = np.histogram(result_t1_angle[(result_t1_angle["gamma"]==0) & (result_t1_angle["repeat"]==repeat)]['angle'], bins=b, range=(0,np.pi), density=False)
    counts0=counts0_/np.sum(counts0_)*100
    polar_ax.bar((bin_0[:-1]+bin_0[1]/2), counts0, width=np.pi/b, alpha=0.5, label='0')

    counts_, bin_ = np.histogram(result_t1_angle[(result_t1_angle["gamma"]==0.2) & (result_t1_angle["repeat"]==repeat)]['angle'], bins=b, range=(0,np.pi), density=False)
    counts = counts_/np.sum(counts_)*100
    polar_ax.bar((bin_[:-1]+bin_[1]/2), counts, width=np.pi/b, alpha=0.5, label='0.2')

    print(np.sum(counts0_), np.sum(counts_))
    fig.legend()
    fig.set_size_inches((10,10))
    fig.savefig(str(SIM_DIR/'angle_new_junction')+str(repeat)+'.png', dpi=150)

In [ ]:
fig = plt.figure()
polar_ax = fig.add_subplot(1, 1, 1, projection="polar")

b=10

# bin data for our polar histogram
counts0_, bin_0 = np.histogram(result_t1_angle[(result_t1_angle["gamma"]==0)]['angle'], bins=b, range=(0,np.pi), density=False)
counts0=counts0_/np.sum(counts0_)*100
polar_ax.bar((bin_0[:-1]+bin_0[1]/2), counts0, width=np.pi/b, alpha=0.5, label='0')

counts_, bin_ = np.histogram(result_t1_angle[(result_t1_angle["gamma"]==0.2)]['angle'], bins=b, range=(0,np.pi), density=False)
counts = counts_/np.sum(counts_)*100
polar_ax.bar((bin_[:-1]+bin_[1]/2), counts, width=np.pi/b, alpha=0.5, label='0.2')

print(np.sum(counts0_), np.sum(counts_))
fig.legend()
fig.set_size_inches((10,10))
# fig.savefig(SIM_DIR/'angle_new_junction.png', dpi=150)

# Result V3

In [ ]:
result = pd.DataFrame(columns = ['repeat', 'gamma', 'id_edge', 'length', 'orient', 'segment'])

repeat = np.arange(10)
gammas = np.linspace(0, 0.2, 21)
# gammas = np.linspace(0.19, 0.19, 1)

for r in repeat:
    sim_save_dir = SIM_DIR/str(r)   
    for g in gammas:
        dir_ = sim_save_dir/str(g)
        try : 
            monolayer_d = load_datasets(os.path.join(sim_save_dir/str(g),'monolayer99.hf5'))
        except: 
            monolayer_d = load_datasets(os.path.join(sim_save_dir/str(g),'monolayer199.hf5'))
        monolayer = Monolayer("mono", monolayer_d)
        id_new_edges = monolayer.edge_df[(monolayer.edge_df['face'].isin(monolayer.face_df[monolayer.face_df['num_sides']==3].index)) 
                                         & (monolayer.edge_df['face'].isin(monolayer.face_df[monolayer.face_df['area']>0.01].index))
                                         &   (((monolayer.edge_df['sz']>0.4) & (monolayer.edge_df['tz']>0.4)) |
                                             ((monolayer.edge_df['sz']<-0.4) & (monolayer.edge_df['tz']<-0.4))) 
                                         ].index

        length = monolayer.edge_df.loc[id_new_edges, 'length']
        dx = monolayer.edge_df.loc[id_new_edges, 'sx'] - monolayer.edge_df.loc[id_new_edges, 'tx']
        dy = monolayer.edge_df.loc[id_new_edges, 'sy'] - monolayer.edge_df.loc[id_new_edges, 'ty']
        orient = np.arctan2(dy, dx)
        
        segment = ['basal' if p<0 else 'apical' for p in monolayer.edge_df.loc[id_new_edges]['sz']]
        
        
        result = pd.concat([result, pd.DataFrame({'repeat':r,
                                                  'gamma':g, 
                                                  'id_edge': id_new_edges,
                                                  'length':length,
                                                  'orient': orient,
                                                  'segment':segment})],
                          ignore_index=True)
        

        
result["angle_0_pi"] = [np.pi+a if a<0 else a for a in result['orient']]

In [ ]:
result_t1_angle = result.copy(deep=True)

In [ ]:
# result_t1_angle.to_csv(os.path.join(SIM_DIR, 'result_t1_angle_v2.csv'))

In [ ]:
plt.hist(result_t1_angle['length'])

In [ ]:
fig = plt.figure()
polar_ax = fig.add_subplot(1, 1, 1, projection="polar")

b=18
l = 0.7
seg = 'basal'
# bin data for our polar histogram
counts0_, bin_0 = np.histogram(result_t1_angle[(result_t1_angle["gamma"]==0) & 
                                               (result_t1_angle["length"]>l)  
                                              & (result_t1_angle["segment"]==seg)
                                              ]['angle_0_pi'],
                               bins=b, range=(0,np.pi), density=False)
counts0=counts0_/np.sum(counts0_)*100
polar_ax.bar((bin_0[:-1]+bin_0[1]/2), counts0, width=np.pi/b, alpha=0.5, label='0')

counts_, bin_ = np.histogram(result_t1_angle[(result_t1_angle["gamma"]==0.2) &
                                             (result_t1_angle["length"]>l)
                                            & (result_t1_angle["segment"]==seg)
                                            ]['angle_0_pi'], 
                             bins=b, range=(0,np.pi), density=False)
counts = counts_/np.sum(counts_)*100
polar_ax.bar((bin_[:-1]+bin_[1]/2), counts, width=np.pi/b, alpha=0.5, label='0.2')

print(np.sum(counts0_), np.sum(counts_))
fig.legend()
fig.set_size_inches((10,10))
fig.savefig(SIM_DIR/'angle_new_junction_basal_l0_7.png', dpi=150)

In [ ]:
b=18
l = 0.8
seg = 'basal'
for repeat in range (2, 10):
    fig = plt.figure()
    polar_ax = fig.add_subplot(1, 1, 1, projection="polar")

    # bin data for our polar histogram
    counts0_, bin_0 = np.histogram(result_t1_angle[(result_t1_angle["gamma"]==0) & 
                                                   (result_t1_angle["length"]>l) & 
                                                   (result_t1_angle['segment']==seg) & 
                                                   (result_t1_angle["repeat"]==repeat)]['angle_0_pi'], bins=b, range=(0,np.pi), density=False)
    counts0=counts0_/np.sum(counts0_)*100
    polar_ax.bar((bin_0[:-1]+bin_0[1]/2), counts0, width=np.pi/b, alpha=0.5, label='0')

    counts_, bin_ = np.histogram(result_t1_angle[(result_t1_angle["gamma"]==0.2) & 
                                                 (result_t1_angle["length"]>l) & 
                                                 (result_t1_angle['segment']==seg) & 
                                                 (result_t1_angle["repeat"]==repeat)]['angle_0_pi'], bins=b, range=(0,np.pi), density=False)
    counts = counts_/np.sum(counts_)*100
    polar_ax.bar((bin_[:-1]+bin_[1]/2), counts, width=np.pi/b, alpha=0.5, label='0.2')

    print(np.sum(counts0_), np.sum(counts_))
    fig.legend()
    fig.set_size_inches((10,10))
#     fig.savefig(str(SIM_DIR/'angle_new_junction')+str(repeat)+'.png', dpi=150)

In [ ]:
r = 0
g = 0.0
nb_changes = []

sim_save_dir = SIM_DIR/str(r)   
dir_ = sim_save_dir/str(g)
for i in range(200):
    monolayer_d = load_datasets(os.path.join(sim_save_dir/str(g),'monolayer'+str(i)+'.hf5'))

    monolayer = Monolayer("mono", monolayer_d)
    count = len(np.unique(monolayer.edge_df[(monolayer.edge_df['face'].isin(monolayer.face_df[monolayer.face_df['num_sides']==3].index)) &
                  (monolayer.edge_df['segment']=='lateral') & (monolayer.edge_df['face'].isin(monolayer.face_df[monolayer.face_df['area']>0.01].index))]['cell']))
    nb_changes.append(count/monolayer.Nc*100)

    
    
tri_face_ids = np.unique(monolayer.edge_df[(monolayer.edge_df['face'].isin(monolayer.face_df[monolayer.face_df['num_sides']==3].index)) &
                  (monolayer.edge_df['segment']=='lateral') & (monolayer.edge_df['face'].isin(monolayer.face_df[monolayer.face_df['area']>0.01].index))]['cell'])

angle0=[]
for f in tri_face_ids :
    dx = monolayer.edge_df[(monolayer.edge_df['face']==f) & 
                    (((monolayer.edge_df['sz']>0.4) & (monolayer.edge_df['tz']>0.4)) |
                     ((monolayer.edge_df['sz']<-0.4) & (monolayer.edge_df['tz']<-0.4)))]['dx'].to_numpy()
    dy = monolayer.edge_df[(monolayer.edge_df['face']==f) & 
                    (((monolayer.edge_df['sz']>0.4) & (monolayer.edge_df['tz']>0.4)) |
                     ((monolayer.edge_df['sz']<-0.4) & (monolayer.edge_df['tz']<-0.4)))]['dy'].to_numpy()
    if len(dx)!=0:
        angle0.append(np.arctan2(dy, dx)[0]*180/np.pi)        
 



        
g = 0.2
nb_changes2 = []

sim_save_dir = SIM_DIR/str(r)   
dir_ = sim_save_dir/str(g)
for i in range(200):
    monolayer_d = load_datasets(os.path.join(sim_save_dir/str(g),'monolayer'+str(i)+'.hf5'))

    monolayer = Monolayer("mono", monolayer_d)
    count = len(np.unique(monolayer.edge_df[(monolayer.edge_df['face'].isin(monolayer.face_df[monolayer.face_df['num_sides']==3].index)) &
                  (monolayer.edge_df['segment']=='lateral') & (monolayer.edge_df['face'].isin(monolayer.face_df[monolayer.face_df['area']>0.01].index))]['cell']))
    nb_changes2.append(count/monolayer.Nc*100)
    
    
tri_face_ids = np.unique(monolayer.edge_df[(monolayer.edge_df['face'].isin(monolayer.face_df[monolayer.face_df['num_sides']==3].index)) &
                  (monolayer.edge_df['segment']=='lateral') & (monolayer.edge_df['face'].isin(monolayer.face_df[monolayer.face_df['area']>0.01].index))]['cell'])

angle02=[]
for f in tri_face_ids :
    dx = monolayer.edge_df[(monolayer.edge_df['face']==f) & 
                    (((monolayer.edge_df['sz']>0.4) & (monolayer.edge_df['tz']>0.4)) |
                     ((monolayer.edge_df['sz']<-0.4) & (monolayer.edge_df['tz']<-0.4)))]['dx'].to_numpy()
    dy = monolayer.edge_df[(monolayer.edge_df['face']==f) & 
                    (((monolayer.edge_df['sz']>0.4) & (monolayer.edge_df['tz']>0.4)) |
                     ((monolayer.edge_df['sz']<-0.4) & (monolayer.edge_df['tz']<-0.4)))]['dy'].to_numpy()
    if len(dx)!=0:
        angle02.append(np.arctan2(dy, dx)[0]*180/np.pi)



In [ ]:
fig, ax = plt.subplots()
ax.plot(nb_changes, label='0')
ax.plot(nb_changes2, label='0.2')
ax.legend()
ax.set_xlabel('\'time\'')
ax.set_ylabel('% of cell (cumulative)')

fig.set_size_inches((10,10))
# fig.savefig(SIM_DIR/'T1-time.png', dpi=150)

In [ ]:
plt.hist(angle0, alpha = 0.4, bins=18)
plt.hist(angle02, alpha = 0.4, bins=18)

In [ ]:
angle0 = [180+a for a in angle0 if a<0]
angle02 = [180+a for a in angle02 if a<0]

In [ ]:
fig = plt.figure()
polar_ax = fig.add_subplot(1, 1, 1, projection="polar")

b=18

# bin data for our polar histogram
counts, bin = np.histogram(angle0, bins=b)
polar_ax.bar((bin[:-1]+bin[1]/2)*np.pi/180, counts, width=np.pi/b, alpha=0.8)

counts, bin = np.histogram(angle02, bins=b)
polar_ax.bar((bin[:-1]+bin[1]/2)*np.pi/180, counts, width=np.pi/b, alpha=0.3)

# Label angles according to convention
# polar_ax.set_xticks([0, np.pi/4, 2*np.pi - np.pi/4])
# polar_ax.set_xticklabels([r'$0$', r'$\pi/4$', r'$-\pi/4$'])
fig.set_size_inches((10,10))